# DIM0549 - Grafos : 1º Unidade


---


## Grupo 5

## Grafos


---



### **Item 1** - *Criação do grafo a partir da lista de adjancências*
A implementação do grafo que utiliza a lista de adjacência como estrutura de dados subjacente foi feita através das classes *ListaAdjacencia* e *VerticeAdjacencia*.
Aqui, contamos com o método *ler_grafo*, responsável por realizar a leitura do grafo descrito no filepath fornecido como
entrada e fazer a construção do grafo. Essa construção consiste em, para cada par de vértices lidos, instanciar os objetos
vértices correspondentes, e inserí-los no grafo via o método *adicionar_vertice*, caso já não pertençam ao grafo. Se já pertencerem, é feita a
criação de novas arestas via a função *adicionar_vizinho* encapsulada na classe *VerticeAdjacencia*.
Maior detalhamento sobre o método de adição será fornecido na resolução do item **9**
Esta implementação da classe já conta, também, com a implementação do método de remoção de vértice ao grafo.
#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada


```
== Lista de Adjacência ==
a -> ['b', 'd']
b -> ['a', 'c']
c -> ['b']
d -> ['a']

```





In [21]:
from collections import Counter
class VerticeAdjacencia:
    def __init__(self, label, vizinhos = None):
        self.label = label
        # self.predecessor = predecessor if predecessor else None
        vizinhos = vizinhos if vizinhos else []
        # TODO : Remover
        self.vizinhos = set(vizinhos)

    def adicionar_vizinho(self, vizinho):
        self.vizinhos.add(vizinho)
        vizinho.vizinhos.add(self)

    def remover_vizinho(self, vizinho):
        self.vizinhos.remove(vizinho)
        vizinho.vizinhos.remove(self)
    def __str__(self):
        return f"Label : {self.label}"

# Nesta implementacao o grafo consistirá em um dict cujos valores sao lists (implementacoes de linkedlista=)
class ListaAdjacencia:
    def __init__(self, nome,  caminho_arquivo : str = None):
        self.lista = dict()
        self.labels = dict()
        self.nome = nome
        self.tamanho = 0
        if caminho_arquivo:
            vertices = self.ler_grafo(caminho_arquivo=caminho_arquivo)
            for vertice in vertices:
                self.adicionar_vertice(vertice)

    def adicionar_vertice(self, novo_vertice): # O(m), tq m eh numero de vertices vizinhos ao novo_vertice JA PRESENTES NO GRAFO
        self.lista[novo_vertice] = set()
        self.labels[novo_vertice.label] = novo_vertice
        # Ver os vizinhos do vertice que vai ser adicionado
        # adicionar o novo vertice a lista de colisao de cada um de seus vizinhos

        for vizinho in novo_vertice.vizinhos:
            # Fazemos a checagem se o a aresta ja foi contabilizada para incrementar o atributo 'tamanho'
            # Para isso, checamos, se apos a insercao do novo vertice nos seus vizinhos, a lista de vizinhos
            # deste irá aumentar de tamanho - já que usamos a estrutura de dados set. Se a lista tem tamanho maior após a
            # inserção, essa ligação entre o novo vértice não havia sido contabilizada anteriormente, e o tamanho do grafo
            # é incrementado.
            vizinho.adicionar_vizinho(novo_vertice)
            self.lista[novo_vertice].add(vizinho)
            if not self.lista.get(vizinho):
                self.lista[vizinho] = set()
            tamanho_antes = len(self.lista[vizinho])
            self.lista[vizinho].add(novo_vertice)
            if tamanho_antes < len(self.lista[vizinho]):
                self.tamanho += 1

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                label1, label2 = linha.strip().split(",")
                if label1.isnumeric():
                    label1 = int(label1)
                    label2 = int(label2)
                return label1, label2

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)
                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeAdjacencia(label=label_2, vizinhos=[vert_1])
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeAdjacencia(label=label_2)
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return vertices.values()
    def remover_vertice(self, label_vertice_a_ser_removido):
        vertice_a_ser_removido = self.labels.get(label_vertice_a_ser_removido, None)
        if not vertice_a_ser_removido:
            return "Vertice nao pertence ao grafo"
        self.lista.pop(vertice_a_ser_removido)  # O(1)
        # while len(vertice_a_ser_removido.vizinhos) > 0:
        #     self.lista[vizinho].remove(vertice_a_ser_removido) # O(1)
        #     self.tamanho =- 1


        for vizinho in vertice_a_ser_removido.vizinhos: # O(m)
            # Gambiarra
            if vizinho not in self.lista:
                continue
            self.lista[vizinho].remove(vertice_a_ser_removido) # O(1)
            self.tamanho =- 1

    def get_vertices(self):
        return self.lista.keys()

    def get_vertice(self,label):
        return self.labels[label]


    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        return set(vertice.vizinhos)

    def __str__(self):
        if len(self.lista) ==0:
            return "<ListaAdjacencia Vazio>"
        strings = [f"{vertice.label} -> {list(map(lambda vizinho : vizinho.label, self.lista[vertice]))}" for vertice in self.lista.keys()]
        return '\n'.join(strings)

In [24]:
print(" == Grafo 0 ==")
grafo = ListaAdjacencia("== Grafo 0 ==",caminho_arquivo="./data/GRAFO_0.txt")
print(grafo)
print(" == Grafo 1 ==")
grafo = ListaAdjacencia("== Grafo 1 ==",caminho_arquivo="./data/GRAFO_1.txt")
print(grafo)
print(" == Grafo 2 ==")
grafo = ListaAdjacencia("== Grafo 2 ==",caminho_arquivo="./data/GRAFO_2.txt")
print(grafo)
print(" == Grafo 3 ==")
grafo = ListaAdjacencia("== Grafo 3 ==",caminho_arquivo="./data/GRAFO_3.txt")
print(grafo)

 == Grafo 0 ==


FileNotFoundError: [Errno 2] No such file or directory: './data/GRAFO_0.txt'

### **Item 2** - *Implementação do Grafo usando Matriz de adjacência*



A implementação usando matriz de adjacência tem a mesma estrutura do item anterior, é feita a leitura via método *ler_grafo* e adição dos vértices.
Nesta implementação, mantemos o a classe Vertice, mas aqui ela é usada apenas para encapsular o dado vértice e seus vizinhos, para que sirva de entrada ao método *adicionar_vertice*. A estrutura do grafo é mantida integralmente na matriz

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada


```
[1, 0, 1]  
[1, 1, 0]
[0, 1, 0]
[0, 0, 1]  

```


In [ ]:
from collections import Counter
from functools import reduce

# Implementacao da classe vertice para de grafos utilizando matriz de adjacencia
class VerticeMatriz:
    def __init__(self, label : str = "", vizinhos = None):
        self.label = label
        self.indice = None
        # O Counter associa um objeto a um contador numérico via hash table, aqui será usado para contar o número de
        # arestas entre este vértice e seu respectivo vizinho, para o caso de recebermos multigrafos
        self.vizinhos = Counter()
        if vizinhos:
            for vizinho in vizinhos:
                self.adicionar_vizinho(vizinho)

    def adicionar_vizinho(self, vertice_vizinho):
        self.vizinhos[vertice_vizinho] += 1
        vertice_vizinho.vizinhos[self] += 1

class MatrizAdjacencia:
    def __init__(self, nome : str ,caminho_arquivo : str = None, vertices = None): # O(n^2)
        self.nome = nome # Nome do grafo, pra proposito de impressao
        self.indices = dict() # hash table associando os vertices a cada indice da matriz, para agilizar consultas
        self.matriz = []
        self.labels = dict() # Hash table para armazenar os vértices associadas a cada label
        self.tamanho = 0
        if vertices:
            for vertice in vertices:
                self.adicionar_vertice(vertice)
        else:
            if caminho_arquivo:
                for vertice in self.ler_grafo(caminho_arquivo):
                    self.adicionar_vertice(vertice)

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                label1, label2 = linha.strip().split(",")
                if label1.isnumeric():
                    label1 = int(label1)
                    label2 = int(label2)
                return label1, label2

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)
                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1.adicionar_vizinho(vert_2)
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return sorted(vertices.values(), key=lambda key : key.label)


    def adicionar_vertice(self, vertice : VerticeMatriz): # O(n)
        # Adiciona o novo label
        self.labels[vertice.label] = vertice
        # Caso a matriz esteja vazia
        if len(self.matriz) == 0:
            vertice.indice = 0
            # Adicionamos à linha o valor referente ao número de laços presentes neste vértice
            self.matriz.append(list([vertice.vizinhos[vertice]]))
            self.indices[0] = vertice
            self.tamanho += vertice.vizinhos[vertice]
            return

        # Linha que será adcioanada à matriz
        nova_linha = []
        # Adiciona o índice do novo vértice à hash table
        self.indices[len(self.matriz)] = vertice # O(1)
        vertice.indice = len(self.matriz) # O(1)
        # Itera por cada linha da matriz adicionando uma nova coluna com valor igual ao número de arestas entre
        # o novo vértice e o da linha atual
        for indice, linha in enumerate(self.matriz): # O(n)
            vertice_atual = self.indices[indice] # O(1)
            quantidade = vertice.vizinhos.get(vertice_atual, 0) # O(1)
            linha.append(quantidade) # O(1)
            nova_linha.append(quantidade) #O(1)
            self.tamanho += quantidade

        # Atualiza a nova linha para o caso do vértice adicionado apresentar laços
        quantidade_arestas_proprias = vertice.vizinhos.get(vertice, 0) # O(1)
        self.tamanho += quantidade_arestas_proprias
        nova_linha.append(quantidade_arestas_proprias) # O(1)
        self.matriz.append(nova_linha) #O(1)

    def get_vertices(self):
        return self.indices.values()

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        indice_na_matriz = vertice.indice
        linha = self.matriz[indice_na_matriz]
        # É feita a filtragem dos elementos para obter os elementos da linha com valor maior que 0,
        # ou seja, elementos no qual existe um ligação
        vizinhos_map = map(lambda i : self.indices[i], filter(lambda indice : linha[indice] > 0, range(0, len(linha))))
        return set(vizinhos_map)

    def get_vertice(self, label_vertice):
        return self.labels[label_vertice]

    def remover_vertice(self, label_vertice): # O(n + n*n) = O(n^2)
        if label_vertice not in self.labels:
            raise ValueError("Vértice não faz parte do grafo") # O(1)

        vertice = self.labels[label_vertice]
        indice_a_remover = vertice.indice
        for i, linha in enumerate(self.matriz): # O(n)
            if i == indice_a_remover:
                continue
            if linha[indice_a_remover] != 0:    # O(1)
                # Remove o vertice removido da lista de vizinhos dos outros vertices
                self.indices[i].vizinhos.pop(vertice) # O(1)
                self.tamanho -= 1
            # Atualiza os valores dos indices associados aos vertices para refletir as novas posicoes
            if i > indice_a_remover:
                self.indices[i - 1] = self.indices[i]
                self.indices[i - 1].indice = i - 1
                self.indices.pop(i)
            linha.pop(indice_a_remover) # O(n)

        # Remove da matriz a linha referente ao vértice removido
        self.matriz.pop(indice_a_remover)

    def __str__(self):
        # for linha in self.matriz:

        return "\n".join(map(str, self.matriz))

In [ ]:
grafo = MatrizAdjacencia("Grafo - 0", "./data/GRAFO_0.txt")
print(" == Grafo 0 ==")
print(grafo)
grafo = MatrizAdjacencia("Grafo - 0", "./data/GRAFO_1.txt")
print(" == Grafo 1 ==")
print(grafo)
grafo = MatrizAdjacencia("Grafo - 0", "./data/GRAFO_2.txt")
print(" == Grafo 2 ==")
print(grafo)
grafo = MatrizAdjacencia("Grafo - 0", "./data/GRAFO_3.txt")
print(" == Grafo 3 ==")
print(grafo)

 == Grafo 0 ==
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 0]
 == Grafo 1 ==
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 1]
[0, 1, 1, 0, 0, 0, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
 == Grafo 2 ==
[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 == Grafo 3 ==
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 

### **Item 3** - *Implementação do Grafo usando da Matriz de Incidência*

A classe MatrizIncidencia serve para representar um grafo não direcionado usando uma matriz de incidência, onde cada linha representa um vértice e cada coluna representa uma aresta. A função principal da classe é ler um grafo a partir de um arquivo, identificar os vértices e arestas, e construir a matriz de incidência correspondente.
#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada


```
[1, 0, 1]  
[1, 1, 0]
[0, 1, 0]
[0, 0, 1]  

```



In [ ]:
class MatrizIncidencia:
    def __init__(self, nome, caminho_arquivo=None):
        self.nome = nome # para melhor identificação
        self.vertices = {}  # hash table para mapear label do vértice para o índice
        self.arestas = []   # hash table representando as arestas
        self.matriz = []

        if caminho_arquivo:
            self.ler_grafo(caminho_arquivo)

    def ler_grafo(self, caminho_arquivo):
        with open(caminho_arquivo, "r") as f:
            linhas = f.readlines()[1:] # para ignorar a primeira linha com o número de vértices

            # Primeiro, identifica todos os vértices únicos para definir o tamanho da matriz
            vertices_labels = set()
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                vertices_labels.add(v1_label)
                vertices_labels.add(v2_label)

            for label in sorted(list(vertices_labels)): # ordena para consistência
                self.adicionar_vertice(label)

            # adiciona as arestas e constrói a matriz
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                self.adicionar_aresta(v1_label, v2_label)

    def adicionar_vertice(self, label):
        if label not in self.vertices:
            self.vertices[label] = len(self.vertices)
            # adiciona uma nova linha (vazia por enquanto) para o novo vértice
            self.matriz.append([])

    def adicionar_aresta(self, v1_label, v2_label):
        # adiciona a nova aresta
        self.arestas.append((v1_label, v2_label))
        indice_aresta = len(self.arestas) - 1

        # adiciona a nova coluna à matriz (representada por adicionar um elemento a cada linha)
        for i in range(len(self.vertices)):
            self.matriz[i].append(0)

        # preenche os valores da nova coluna
        i1 = self.vertices[v1_label]
        i2 = self.vertices[v2_label]

        if i1 == i2:  # laço
            self.matriz[i1][indice_aresta] = 2
        else:  # grafo não direcionado
            self.matriz[i1][indice_aresta] = 1
            self.matriz[i2][indice_aresta] = 1

    def __str__(self):
       return str(self.matriz)

grafo_incidencia = MatrizIncidencia("GRAFO 1", "./data/GRAFO_1.txt")
print(grafo_incidencia)

[[1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0]]


### **Item 4** - *Conversão de matriz de adjacência para lista de Adjacências e vice-versa*


As funções converter_lista_pra_matriz e converter_matriz_pra_lista realizam a conversão entre as duas principais representações de grafos: lista de adjacência e matriz de adjacência.

* converter_lista_pra_matriz: Cria um novo grafo em matriz, adicionando os vértices e suas conexões baseadas na lista de adjacência do grafo original.

* converter_matriz_pra_lista: Cria um novo grafo em lista, inserindo cada vértice da matriz no novo grafo.

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada
* Grafo Original:
```
[0, 1, 0, 1]  # a -> b, d
[0, 0, 1, 0]  # b -> c
[0, 0, 0, 0]  # c -> nenhum
[1, 0, 0, 0]  # d -> a
```
* Convertido para lista de adjacência
```
a -> ['b', 'd']
b -> ['c']
c -> []
d -> ['a']

```

In [ ]:
def converter_lista_pra_matriz(grafo_inicial : ListaAdjacencia) -> MatrizAdjacencia:
    """
    Converte um grafo representado por lista de adjacência para matriz de adjacência.
    - Para cada vértice do grafo inicial, cria-se um vértice correspondente na matriz.
    - Para cada vizinho do vértice, cria-se ou recupera-se o vértice correspondente e adiciona-se como adjacente.
    - Ao final, cria-se uma instância de MatrizAdjacencia com todos os vértices convertidos.
    """
    vertices_convertidos = dict()
    for vertice in grafo_inicial.get_vertices():
        vertice_convertido = VerticeMatriz(label=vertice.label)
        for vizinho in vertice.vizinhos:
            vizinho_convertido = vertices_convertidos.get(vizinho.label, None)
            if not vizinho_convertido:
                vizinho_convertido = VerticeMatriz(label=vizinho.label)
                vertices_convertidos[vizinho.label] = vizinho_convertido
            vertice_convertido.adicionar_vizinho(vizinho_convertido)
            # vizinho_convertido.adicionar_vizinho(vertice_convertido)
        vertices_convertidos[vertice_convertido.label] = vertice_convertido
    # Cria o grafo final em matriz de adjacência, ordenando os vértices pelo label
    novo = MatrizAdjacencia(nome=grafo_inicial.nome, vertices=sorted(list(vertices_convertidos.values()), key=lambda key : key.label))
    return  novo

def converter_matriz_pra_lista(grafo_inicial : MatrizAdjacencia) -> ListaAdjacencia:
    """
    Converte um grafo representado por matriz de adjacência para lista de adjacência.
    - Para cada vértice da matriz, adiciona-se um vértice correspondente na lista.
    - As conexões já estão mantidas pelo objeto Vertice, portanto não é necessário recriar vizinhos.
    """
    novo_grafo = ListaAdjacencia(grafo_inicial.nome)
    for vertice in grafo_inicial.get_vertices():
        novo_grafo.adicionar_vertice(vertice)
    return novo_grafo

print("== Grafo 0 ==")
matriz = MatrizAdjacencia(nome="Grafo que sera convertido", caminho_arquivo="./data/GRAFO_0.txt")
print(matriz)
lista = converter_matriz_pra_lista(matriz)

print("== Grafo 0 convertido de matriz para lista de adjacencia ==")
print(converter_matriz_pra_lista(matriz))
matriz = converter_lista_pra_matriz(lista)
print("== Grafo 0 convertido de lista para matriz de adjacencia ==")
print(converter_lista_pra_matriz(lista))


== Grafo 0 ==
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 0]
== Grafo 0 convertido de matriz para lista de adjacencia ==
a -> ['b']
b -> ['a', 'd', 'c']
c -> ['b', 'd', 'e']
d -> ['b', 'c']
e -> ['c', 'f']
f -> ['h', 'g', 'e']
g -> ['h', 'f']
h -> ['g', 'f']
== Grafo 0 convertido de lista para matriz de adjacencia ==
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1, 0]


### **Item 5** - *Função que calcula o grau de cada vértice*

A função calcular_graus determina o grau de cada vértice em um grafo. O grau representa o número de vizinhos (adjacentes) de cada vértice. Funciona tanto para lista de adjacência quanto para matriz de adjacência.
#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada

```
Graus dos Vértices do 'Grafo 0'
Grau(a) = 1
Grau(b) = 1
Grau(c) = 0
Grau(d) = 1
```




In [ ]:
def calcular_graus(grafo):

    graus = {} # para armazenar os graus dos vértices

    for vertice in grafo.get_vertices():
        # o grau é o número de vizinhos (adjacentes)
        grau = len(grafo.get_adjacentes(vertice.label)) # conta o número de elementos na lista
        graus[vertice.label] = grau

    return graus

grafo_exemplo = ListaAdjacencia("Grafo 2", "./data/GRAFO_2.txt")

# calculando os graus de todos os vértices
graus_dos_vertices = calcular_graus(grafo_exemplo)

# exibe os resultados
print(f" Graus dos Vértices do '{grafo_exemplo.nome}'")
for vertice, grau in sorted(graus_dos_vertices.items()):
    print(f"Grau({vertice}) = {grau}")

 Graus dos Vértices do 'Grafo 2'
Grau(1) = 3
Grau(2) = 4
Grau(3) = 3
Grau(4) = 2
Grau(5) = 2
Grau(6) = 3
Grau(7) = 1
Grau(8) = 3
Grau(9) = 2
Grau(10) = 2
Grau(11) = 1


### **Item 6** - *Função que determina se dois vértices são adjacentes*

A função sao_adjacentes verifica se dois vértices de um grafo são adjacentes (existe uma aresta ligando-os), funcionando tanto para lista de adjacência quanto para matriz de adjacência.
#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada


```
Os vertices a e b sao adjacentes? (espera-se True): True

Os vertices b e d sao adjacentes? (espera-se False): False
```



In [ ]:
# Implementacao para a Matriz e Lista de adjacencia
def sao_adjacentes(grafo, label_vert_1, label_vert_2) -> bool : # O(1)
    vert_1, vert_2 = grafo.get_vertice(label_vert_1) , grafo.get_vertice(label_vert_2) # O(1)
    return vert_2 in vert_1.vizinhos or vert_1 in vert_2.vizinhos


grafo = MatrizAdjacencia("Grafo", "./data/GRAFO_2.txt")
print(f"Os vertices 1 e 6 sao adjacentes? (espera-se True): {sao_adjacentes(grafo, 1, 6)}")
print(f"Os vertices 2 e 9 sao adjacentes? (espera-se False): {sao_adjacentes(grafo, 2, 9)}")


Os vertices 1 e 6 sao adjacentes? (espera-se True): True
Os vertices 2 e 9 sao adjacentes? (espera-se False): False


### **Itens 7 e 8** - *Função que determina os números totais de vértices e arestas*


Para obter a ordem dos grafos implementados com lista de adjacência, basta obter o tamanho do seu atributo *lista*, já para a matriz, o tamanho de *matriz*.
A obtenção do tamanho para os grafos nessas estruturas é um pouco mais complexa. A solução ingênua seria: para a lista, iterar por cada vértice e incrementar o tamanho para cada vizinho encontrado, enquanto o para a matriz basta checar as arestas em cada linha.
Para dirimir a complexidade de invocar este método, o atributo tamanho é ajustado a cada chamada dos métodos *adicionar_vertice* e *remover_vertice*. Ou seja, calculamos o tamanho ao criar a árvore e o atualizamos ao mudar sua estrutura.
Para a lista, ao adicionar vértice, é checado se as novas arestas conectadas a ele já foram contabilizadas - no caso de seu vértice adjacente conectado a esta aresta já ter sido adicionado - se não, incrementamos o atributo *tamanho*.
####Entrada esperada:
Um arquivo .txt


```
a,b
b,c
d,a

```

####Saída esperada:
```
== Grafo 0 (Matriz de Adjacencia) ==
Ordem do grafo : 4   # vértices: a, b, c, d
Tamanho do grafo : 3  # arestas: (a,b), (b,c), (d,a)

```

In [ ]:
# Função para obter o tamanho (número de arestas) do grafo
def get_tamanho(grafo):
    return grafo.tamanho

# Funções para obter a ordem (número de vértices)
def get_ordem_lista(grafo):
    # Para lista de adjacência, retorna o número de chaves
    return len(grafo.lista)

def get_ordem_matriz_adjacencia(grafo):
    # Para matriz de adjacência, retorna número de linhas
    return len(grafo.matriz)

# Adicionando os métodos às classes
MatrizAdjacencia.get_ordem = get_ordem_matriz_adjacencia
ListaAdjacencia.get_ordem = get_ordem_lista


MatrizAdjacencia.get_tamanho = get_tamanho
ListaAdjacencia.get_tamanho = get_tamanho
grafo = ListaAdjacencia("Grafo 0 - Lista de Adjacencia", "./data/GRAFO_0.txt")
print(" == Grafo 0 (Lista de Adjacencia) == ")
print(f"Ordem do grafo : {grafo.get_ordem()}")
print(f"Tamanho do grafo : {grafo.get_tamanho()}")

print(" == Grafo 0 (Matriz de Adjacencia) == ")
grafo_mat = MatrizAdjacencia("Grafo 0 - Matriz de Adjacencia", "./data/GRAFO_0.txt")
print(f"Ordem do grafo : {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo : {grafo_mat.get_tamanho()}")
print(" == Grafo 1 (Lista de Adjacencia) == ")
grafo = ListaAdjacencia("Grafo 1 - Lista de Adjacencia", "./data/GRAFO_1.txt")
print(f"Ordem do grafo : {grafo.get_ordem()}")
print(f"Tamanho do grafo : {grafo.get_tamanho()}")

print(" == Grafo 1 (Matriz de Adjacencia) == ")
grafo_mat = MatrizAdjacencia("Grafo 1 - Matriz de Adjacencia", "./data/GRAFO_1.txt")
print(f"Ordem do grafo : {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo : {grafo_mat.get_tamanho()}")

print(" == Grafo 2 (Lista de Adjacencia) == ")
grafo = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
print(f"Ordem do grafo : {grafo.get_ordem()}")
print(f"Tamanho do grafo : {grafo.get_tamanho()}")

print(" == Grafo 2 (Matriz de Adjacencia) == ")
grafo_mat = MatrizAdjacencia("Grafo 2 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
print(f"Ordem do grafo : {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo : {grafo_mat.get_tamanho()}")

print(" == Grafo 3 (Lista de Adjacencia) == ")
grafo = ListaAdjacencia("Grafo 3 - Lista de Adjacencia", "./data/GRAFO_3.txt")
print(f"Ordem do grafo : {grafo.get_ordem()}")
print(f"Tamanho do grafo : {grafo.get_tamanho()}")

print(" == Grafo 3 (Matriz de Adjacencia) == ")
grafo_mat = MatrizAdjacencia("Grafo 3 - Matriz de Adjacencia", "./data/GRAFO_3.txt")
print(f"Ordem do grafo : {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo : {grafo_mat.get_tamanho()}")


 == Grafo 0 (Lista de Adjacencia) == 
Ordem do grafo : 8
Tamanho do grafo : 9
 == Grafo 0 (Matriz de Adjacencia) == 
Ordem do grafo : 8
Tamanho do grafo : 9
 == Grafo 1 (Lista de Adjacencia) == 
Ordem do grafo : 8
Tamanho do grafo : 9
 == Grafo 1 (Matriz de Adjacencia) == 
Ordem do grafo : 8
Tamanho do grafo : 9
 == Grafo 2 (Lista de Adjacencia) == 
Ordem do grafo : 11
Tamanho do grafo : 13
 == Grafo 2 (Matriz de Adjacencia) == 
Ordem do grafo : 11
Tamanho do grafo : 13
 == Grafo 3 (Lista de Adjacencia) == 
Ordem do grafo : 17
Tamanho do grafo : 24
 == Grafo 3 (Matriz de Adjacencia) == 
Ordem do grafo : 17
Tamanho do grafo : 24


### **Itens 9 e 10** - *Inclusão e exclusão de vértices em matriz de adjacencia e lista de adjacencia*


As funções para inclusão e exclusão de vértices nestas representações de grafo foram declaradas junto das classes dos grafos.
A adição de novos vértices é usada nos construtores de ambas as implementações. Para a lista de adjacência, o método - que recebe um argumento de tipo *VerticeAdjacencia* - faz uma inserção na lista principal, que, neste trabalho, foi implementada como um hash map; e o em seguida atualiza as listas de vizinhos de cada vértice adjacente a este novo.

Para a matriz de adjacência, é criada uma nova linha que é adicionada à matriz e, para cada linha anterior, é adicionado um novo elemento de valor 0 - caso não seja adjacente ao vértice adicionado - ou 1 - caso contrário.

A remoção de vértices segue a lógica inversa: para a lista de adjacência, simplesmente retira-se a chave vértice da lista principal e iteramos por cada vértice vizinho do removido para atualizar a sua lista de vértices adjacentes.

Já para a matriz, a linha correspondente é removida e para cada outra linha, o elemento na posição referente ao vértice retirado também é removido (já que usamos uma lista simples como linha, essa operação acaba tendo complexidade de pior caso na ordem de O(n), para n sendo a ordem do grafo).

Para a matriz, há a complexidade adicional de, durante a remoção, atualizar as estruturas auxiliares usadas para agilizar as operações (atualizar o dicionário de índices, por exemplo).

Abaixo segue uma demonstração dos métodos para remoção de vértice, já que a adição já pode ser vista no processo de criação de um grafo.

####Entrada esperada:
Um arquivo .txt


```
a,b
b,c
d,a

```

####Saída esperada:
* Lista de adjacência
```
Grafo inicial:
a -> ['b']
b -> ['c']
c -> []
d -> ['a']

Removendo vértice a:
b -> ['c']
c -> []
d -> []

Removendo vértice b:
c -> []
d -> []

Removendo vértice c:
d -> []

Removendo vértice d:
<ListaAdjacencia Vazio>

```
* Matriz de adjacência
```
Grafo inicial:
[0, 1, 0, 0]  # a
[0, 0, 1, 0]  # b
[0, 0, 0, 0]  # c
[1, 0, 0, 0]  # d

Removendo vértice a:
[0, 1, 0]  # b
[0, 0, 0]  # c
[0, 0, 0]  # d

Removendo vértice b:
[0, 0]  # c
[0, 0]  # d

Removendo vértice c:
[0]  # d

Removendo vértice d:
[]

```

In [25]:
grafo = ListaAdjacencia("Grafo 0", "./data/GRAFO_0.txt")
print(" == Grafo 0 (Lista de Adjacencia) ==")
print(grafo)

# Percorre e remove cada vértice, mostrando o grafo após cada remoção
for vertice in sorted(grafo.get_vertices(), key=lambda vertice : vertice.label):
    print(f"Removendo vértice {vertice.label}:")
    grafo.remover_vertice(vertice.label)
    print(grafo)

print("\n")
grafo = MatrizAdjacencia("Grafo 0", "./data/GRAFO_0.txt")
print(" == Grafo 1 (Matriz de Adjacencia) ==")
print(grafo)

# Percorre e remove cada vértice, mostrando a matriz após cada remoção
for vertice in sorted(grafo.get_vertices(), key=lambda vertice : vertice.label):
    print(f"Removendo vértice {vertice.label}:")
    grafo.remover_vertice(vertice.label)
    print(grafo)

FileNotFoundError: [Errno 2] No such file or directory: './data/GRAFO_0.txt'

### **Item 11** - *Função que determina se um grafo é conexo ou não*

Para implementar essa função, foi utilizada a estratégia de percorrer todos os vértices de um grafo a partir de um vértice inicial, usando uma fila baseada em lista (similar à busca em largura – BFS).

* Se todos os vértices forem visitados durante o percurso, o grafo é conexo.

* Caso contrário, o grafo é desconexo.
####Entrada esperada:
* Objeto Grafo
* O grafo pode ser vazio, mas nesse caso a função retorna False
* Arquivo de grafo de exemplo (.txt) com as arestas no formato:


```
a,b
b,c
d,a

```

####Saída esperada:
O Grafo 1 é conexo? False



In [20]:
def eh_conexo_lista(grafo):
    """
    Verifica se um grafo é conexo usando lista como fila.
    """
    todos_os_vertices = list(grafo.get_vertices())
    if not todos_os_vertices:
        return False  # grafo vazio n é conexo

    fila = [todos_os_vertices[0]]  # lista
    visitados = {todos_os_vertices[0]} # armazena os vertices visitados

    while fila:
        vertice_atual = fila.pop(0)  # remove do inicio da lista
        for vizinho in grafo.get_adjacentes(vertice_atual.label): # se o vizinho ainda n foi visitado, adiciona a fila /conjunto

            if vizinho not in visitados:
                visitados.add(vizinho)
                fila.append(vizinho)

    return len(visitados) == len(todos_os_vertices)  # caso todos os vertices forem visitados, o grafo é conexo

def imprimir_conexo(grafo):
  """
  Verifica se o grafo é conexo e imprime a mensagem correspondente usando o nome do grafo.
  """
  conexo = eh_conexo_lista(grafo)
  status = "Conexo" if conexo else "Desconexo"
  print(f"O Grafo {grafo.nome} é conexo? {conexo}")

grafo_conexo = ListaAdjacencia("Grafo 1", "./data/GRAFO_1.txt")
grafo_desconexo = ListaAdjacencia("Grafo 2", "./data/GRAFO_2.txt")

imprimir_conexo(grafo_conexo)
imprimir_conexo(grafo_desconexo)

NameError: name 'ListaAdjacencia' is not defined

### **Item 12** (Opcional) - *Determinar se um grafo é bipartido*

Para implementar essa função, foi usada a estratégia extraída da seção *Testing bipartiteness* no endereço *https://en.wikipedia.org/wiki/Bipartite_graph*.

O procedimento consiste em realizar um percurso em profundidade (DFS) colorindo o grafo em duas cores (representadas por 1 e -1). Se for possível colorir cada vértice com a cor oposta do seu vértice pai na árvore, o grafo é bipartido. Caso contrário, o grafo não é bipartido.

####Entrada esperada:
* Objeto grafo
* O grafo deve ter pelo menos um vértice.
* Arquivo de grafo de exemplo (.txt) com as arestas:


```
# a,b
b,c
d,a

```

####Saída esperada:
* True: se o grafo for bipartido
* False: se o grafo não é bipartido

In [ ]:
def eh_bipartido(grafo):
    # Pega o vertice inicial
    vertice_inicial = list(grafo.get_vertices())[0]
    pilha = list()
    # Map para armazenar a cor de cada grafo
    vertices_coloridos = dict()
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    # Empilha o vértice inicial
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vertices_coloridos[vertice_inicial.label] = 1
    vizinhos_vertices = dict()
    # TODO : Essa variável é utilizada para armazenar a cor do último vértice visitado, para que em casos de grafos não
    # conexos, a nova raíz tenha a cor oposta. Não sei se isso funciona ou se é necessário.
    ultima_cor = 1 # Usado para grafos nao conexos
    while vertices_nao_explorados:
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            vertices_nao_visitados.remove(nova_raiz)
            # Colore o grafo com a cor oposta à última cor
            vertices_coloridos[nova_raiz.label] = -ultima_cor
        u = pilha[-1]
        if u not in vizinhos_vertices:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]

        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            # Se não foi visitado, pinta com a cor oposta ao pai
            vertices_coloridos[v.label] = -vertices_coloridos[u.label]
            vertices_nao_visitados.remove(v)
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    # Se ja foi visitado e a sua cor não é compatível com a do vértice que se conecta a ele,
                    #  não é bipartido
                    if vertices_coloridos[u.label] != -vertices_coloridos[v.label] :
                        return False
    return True # Se foi possível fazer toda a coloração, o grafo é bipartido


In [ ]:
grafo = ListaAdjacencia("Grafo 0", "./data/GRAFO_0.txt")
grafo1 = MatrizAdjacencia("Grafo 1", "./data/GRAFO_1.txt")
grafo2 = ListaAdjacencia("Grafo 2", "./data/GRAFO_2.txt")
grafo3 = MatrizAdjacencia("Grafo 3", "./data/GRAFO_3.txt")
grafo_bipartido = ListaAdjacencia("Grafo Bipartido", "./data/GRAFO_BIPARTIDO.txt")
grafo_bipartido_2 = ListaAdjacencia("Grafo Bipartido", "./data/GRAFO_BIPARTIDO_2.txt")
grafo_bipartido_disconexo = ListaAdjacencia("Grafo Bipartido", "./data/GRAFO_BIPARTIDO_DISCONEXO.txt")
print(f"O grafo 0 eh bipartido: {eh_bipartido(grafo)}")
print(f"O grafo 1 eh bipartido: {eh_bipartido(grafo1)}")
print(f"O grafo 2 eh bipartido: {eh_bipartido(grafo2)}")
print(f"O grafo 3 eh bipartido: {eh_bipartido(grafo3)}")
print(f"O 'grafo bipartido' eh bipartido: {eh_bipartido(grafo_bipartido)}")
print(f"O 'grafo bipartido_2' eh bipartido: {eh_bipartido(grafo_bipartido_2)}")
print(f"O 'grafo bipartido_disconexo' eh bipartido: {eh_bipartido(grafo_bipartido_disconexo)}")

O grafo 0 eh bipartido: False
O grafo 1 eh bipartido: False
O grafo 2 eh bipartido: False
O grafo 3 eh bipartido: False
O 'grafo bipartido' eh bipartido: True
O 'grafo bipartido_2' eh bipartido: True
O 'grafo bipartido_disconexo' eh bipartido: True


### **Item 13** - *Busca em Largura, a partir de um vértice específico*



Busca em largura inciada a partir de um vértice específico. Tal como a implementação a busca em profundidade, o percurso será feito em  ordem ascendente caso o argumento **ordenado** seja **True**
####Entrada esperada:
*   Objeto Grafo
*   label_vertice_inicial
* Arquivo de grafo de exemplo (.txt) com as arestas no formato:
```
a,b
b,c
d,a
```
####Saída esperada:
Impressão do percurso DFS, indicando os vértices visitados e seus predecessores, além das arestas de retorno:


```
## Percurso em profundidade em 'Grafo Exemplo'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor: X
Arestas de retorno: []
```





In [ ]:
from collections import deque
def width_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    """
    Realiza busca em profundidade (DFS) em grafos não conexos a partir de um vértice inicial.
    """
    fila = deque()
    vertices_nao_visitados = set(grafo.get_vertices()) # Pra o caso do grafo nao ser conexo
    arestas_visitadas = set()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    print("## Percurso em Largura")
    print("Vertice inicial: ", vertice_inicial.label)
    fila.append(vertice_inicial)
    vizinhos_vertices = dict()
    eh_conexo = False
    while len(vertices_nao_visitados) > 0:
        if len(fila) == 0:
            nova_raiz =vertices_nao_visitados.pop()
            print(f"Vertice visitado: {nova_raiz.label}; Predecessor : X",)
            fila.append(nova_raiz)
            eh_conexo = True
        u = fila.popleft()
        # Se os vertices adjacentes de u nao estiverem em nosso dict que os armazena, os insere
        if ordenado:
            labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
            vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
        else:
            vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        while v: # O(n) -> n : numero de vizinhos do vertice u
            if v in vertices_nao_visitados:
                vertices_nao_visitados.remove(v)
                fila.append(v)
                print(f"Vertice visitado: {v.label}; Predecessor : {u.label}",)
            v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None

In [ ]:
grafo_matriz = ListaAdjacencia("Grafo Matriz", "./data/GRAFO_2.txt")
print(grafo_matriz)
width_first_search(grafo_matriz, 1)

1 -> [6, 3, 2]
6 -> [1, 7, 2]
2 -> [1, 6, 3, 5]
3 -> [1, 2, 4]
5 -> [2, 4]
4 -> [3, 5]
7 -> [6]
8 -> [11, 9, 10]
9 -> [8, 10]
10 -> [9, 8]
11 -> [8]
## Percurso em Largura
Vertice inicial:  1
Vertice visitado: 6; Predecessor : 1
Vertice visitado: 3; Predecessor : 1
Vertice visitado: 2; Predecessor : 1
Vertice visitado: 7; Predecessor : 6
Vertice visitado: 4; Predecessor : 3
Vertice visitado: 5; Predecessor : 2
Vertice visitado: 8; Predecessor : X
Vertice visitado: 11; Predecessor : 8
Vertice visitado: 9; Predecessor : 8
Vertice visitado: 10; Predecessor : 8


### **Item 14** - *Busca em Profundidade, com determinação de arestas de retorno, a partir de um vértice em específico*


A função a seguir faz a busca em profundidade a partir de um vértice específico, fazendo o percurso incluindo todos os vértices mesmo em grafo não conexos. O argumento *ordenado* garante que a escolha dos vértices será feita por ordem crescente - com o custo de ordenar os vértices. Se não for inserido, a escolha é aleatória.
#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Sáida esperada
O algoritmo visita os vértices em profundidade, começando em a.


```
## Percurso em profundidade em 'Grafo DFS'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : a
Arestas de retorno: []
```





In [ ]:
def depth_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    pilha = list()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    print(f"## Percurso em profundidade em '{grafo.nome}'")
    print(f"Vertice visitado: {vertice_inicial.label}, predecessor: X", )
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vizinhos_vertices = dict()
    while vertices_nao_explorados:
        # Se a pilha esta vazia mais ainda ha vertices nao explorados, tomamo uma nova raiz
        # e prosseguimos o percurso
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            print(f"Vertice visitado: {nova_raiz.label}, predecessor: X", )
            vertices_nao_visitados.remove(nova_raiz)
        u = pilha[-1]
        if u not in vizinhos_vertices:
            if ordenado:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)

        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            vertices_nao_visitados.remove(v)
            print(f"Vertice visitado: {v.label}, predecessor : {u.label}", )
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjacente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    print("Nova aresta de retorno: ", (u.label,v.label))

    # Printa arestas de retorno
    arestas_de_retorno = map(lambda aresta : (aresta[0].label, aresta[1].label), arestas_de_retorno)
    print(f"Arestas de retorno: {list(arestas_de_retorno)}")

In [ ]:
from IPython.display import display_markdown

grafo = ListaAdjacencia("Grafo 0", "./data/GRAFO_0.txt")
depth_first_search(grafo=grafo, label_vertice_inicial="a", ordenado=True)
print("\n")
grafo = MatrizAdjacencia("Grafo 1", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo, label_vertice_inicial="a", ordenado=True)
print("\n")
grafo = MatrizAdjacencia("Grafo 2", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo, label_vertice_inicial=1, ordenado=True)
print("\n")
grafo = MatrizAdjacencia("Grafo 3", "./data/GRAFO_3.txt")
depth_first_search(grafo=grafo, label_vertice_inicial='a', ordenado=True)



## Percurso em profundidade em 'Grafo 0'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: e, predecessor : c
Vertice visitado: f, predecessor : e
Vertice visitado: g, predecessor : f
Vertice visitado: h, predecessor : g
Nova aresta de retorno:  ('h', 'f')
Arestas de retorno: [('h', 'f'), ('d', 'b')]


## Percurso em profundidade em 'Grafo 1'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor : d
Vertice visitado: e, predecessor : c
Nova aresta de retorno:  ('e', 'b')
Vertice visitado: i, predecessor : c
Vertice visitado: f, predecessor : b
Arestas de retorno: [('e', 'b'), ('d', 'b')]


## Percurso em profundidade em 'Grafo 2'
Vertice visitado: 1, predecessor: X
Vertice visit

In [ ]:

display_markdown("**Busca em profundidade em grafo nao conexo**")
grafo1 = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

grafo1 = MatrizAdjacencia("Grafo 2 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

## Percurso em profundidade em 'Grafo 2 - Lista de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  (5, 2)
Vertice visitado: 6, predecessor : 2
Nova aresta de retorno:  (6, 1)
Vertice visitado: 7, predecessor : 6
Vertice visitado: 8, predecessor: X
Vertice visitado: 9, predecessor : 8
Vertice visitado: 10, predecessor : 9
Nova aresta de retorno:  (10, 8)
Vertice visitado: 11, predecessor : 8
Arestas de retorno: [(10, 8), (6, 1), (3, 1), (5, 2)]
## Percurso em profundidade em 'Grafo 2 - Matriz de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  (5, 2)
Vertice visitado: 6, predecessor : 

### **Item 15** - *Determinar articulações e blocos de biconectividade usando lowpt*

O presente trecho de código tem como objetivo executar uma busca em profundidade (DFS) em um dígrafo, calculando os lowpts de cada vértice, identificando vértices articulação e determinando blocos biconexos.
O algoritmo utiliza pilha para simular recursão e segue o procedimento teórico apresentado em sala de aula sobre DFS e classificação de vértices em componentes conexos.
#### Entrada esperada
Arquivo do tipo .txt ou lista de adjacência equivalente:
```
a,b
b,c
d,a
```

#### Sáida esperada
Percurso em profundidade com lowpts, arestas de retorno, articulações e blocos biconexos:


```
## Percurso em profundidade em 'Grafo - Lista de Adjacência'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor: a
Vertice visitado: c, predecessor: b
Vértice c explorado -> lowpt: c
Vértice b explorado -> lowpt: b
Vértice a explorado -> lowpt: a
Vertice visitado: d, predecessor: X
Nova aresta de retorno: (d, a)
Vértice d explorado -> lowpt: a
Arestas de retorno: [(d, a)]

=== RESULTADOS FINAIS ===
Demarcadores: [d, a, b]
Articulações: [a]

=== BLOCOS BICONEXOS ===
Bloco 1: [d, a]
Bloco 2: [a, b, c]

Vértice a (pai: X) -> lowpt: a
Vértice b (pai: a) -> lowpt: b
Vértice c (pai: b) -> lowpt: c
Vértice d (pai: X) -> lowpt: a
```



####  Função DFS com Lowpt

In [ ]:
def dps_lowpt(grafo, label_vertice_inicial, ordenado: bool = None):
    """
    Executa DFS em um dígrafo para calcular lowpts, vértices articulação
    e blocos biconexos.
    """
    pilha = list()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    # Dicionário de pais (vértice -> pai)
    pais = {}
    # Dicionário de lowpts (vértice -> lowpt)
    lowpts = {}
    # Set de demarcadores
    demarcadores = set()
    # Dicionario de raizes de raizes e qnt de filhos (raiz -> qnt_filhos)
    raizes = {}
    # Set de articulacoes
    articulacoes = set()
    # Determinação dos blocos
    pilha_blocos = []
    blocos_biconexos = []
    print(f"## Percurso em profundidade em '{grafo.nome}'")
    print(f"Vertice visitado: {vertice_inicial.label}, predecessor: X", )
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    pais[vertice_inicial] = None
    lowpts[vertice_inicial] = vertice_inicial
    raizes[vertice_inicial] = 0
    vizinhos_vertices = dict()
    while vertices_nao_explorados:
        if len(pilha) == 0:
            # Novo componente conexo
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(menor_label)
            pilha.append(nova_raiz)
            print(f"Vertice visitado: {nova_raiz.label}, predecessor: X")
            vertices_nao_visitados.remove(nova_raiz)
            pais[nova_raiz] = None
            lowpts[nova_raiz] = nova_raiz
            raizes[nova_raiz] = 0

        u = pilha[-1]

        if u not in vizinhos_vertices:
            if ordenado:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)

        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None

        if not v:
            # Vértice u será removido da pilha - CALCULAR LOWPT AQUI
            pilha.pop()
            vertices_nao_explorados.remove(u)

            # Inicialmente, lowpt é o próprio vértice u
            current_lowpt = u

            # Verifica se há arestas de retorno que saem do vértice
            for aresta in arestas_de_retorno:
                if aresta[0] == u:
                    current_lowpt = aresta[1]  # vértice alcançado pela aresta de retorno é o novo lowpt

            # Verifica lowpts dos filhos de u
            for w, pai_w in pais.items():
                if pai_w == u: # Se achar um filho de u
                    pos_new = None # posição na pilha do novo possivel lowpt
                    pos_current = len(pilha) + 1 # posição na pilha do atual lowpt (inicializa com o valor maximo possivel para caso o lowpt seja ele mesmo, caso não seja, será tratado durante a iteração)
                    for i, vertice in enumerate(pilha): # percorre a pilha
                        if vertice == lowpts[w] and vertice != w:
                            # caso o vertice na pilha seja o lowpt de w (filho de u) e seja diferente do proprio w, pos_new sera o indice (local na pilha)
                            pos_new = i
                        if vertice == current_lowpt:
                            # caso ache o atual lowpt na pilha (ou seja, o lowpt atual é diferente do proprio vertice) adiciona a posição correta
                            pos_current = i
                    if pos_new is not None: # Checa se achou um possivel candidato para novo lowpt
                        if pos_new < pos_current: # Se achou, checa se o novo candidato está mais em baixo na pilha que o atual
                            current_lowpt = lowpts[w] # Caso esteja, o novo lowpt sera o lowpt do filho

            lowpts[u] = current_lowpt

            if lowpts[u] == u or lowpts[u] == pais[u]: # Checa se u é demarcador
                demarcadores.add(u)

            if u in demarcadores and pais[u] is not None: # Se u for demarcador e não for raiz (tem pai)
                if pais[u] not in raizes: # Checa se o pai é raiz
                    articulacoes.add(pais[u]) # Caso não seja, adiciona o pai como articulação
                bloco = set()
                while pilha_blocos: # Percore a pilha de arestas adicionando os vertices da subarvore
                    x, y = pilha_blocos.pop()
                    bloco.add(x)
                    bloco.add(y)
                    if (x, y) == (u, pais[u]) or (y, x) == (u, pais[u]): # Se encontra a aresta de u com seu pai, termina
                        break
                blocos_biconexos.append(bloco)
            elif u in raizes: # Checa se u é raiz e tem mais de um filho
                if raizes[u] >= 2:
                    articulacoes.add(u) # Caso sim, adiciona como articulacao

            print(f"Vértice {u.label} explorado -> lowpt: {current_lowpt.label}")

        elif v in vertices_nao_visitados:
            vertices_nao_visitados.remove(v)
            print(f"Vertice visitado: {v.label}, predecessor: {u.label}")
            pilha.append(v)
            arestas_visitadas.add((u, v))
            pilha_blocos.append((u, v))
            pais[v] = u
            if u in raizes:
                raizes[u] += 1
            lowpts[v] = v

        else:
            if v in vertices_nao_explorados and (len(pilha) < 2 or pilha[-2] != v):
                if (v, u) not in arestas_visitadas:
                    arestas_visitadas.add((u, v))
                    arestas_de_retorno.add((u, v))
                    pilha_blocos.append((u, v))
                    print("Nova aresta de retorno: ", (u.label, v.label))

    # Printa arestas de retorno
    arestas_de_retorno_labels = [(aresta[0].label, aresta[1].label) for aresta in arestas_de_retorno]
    print(f"Arestas de retorno: {list(arestas_de_retorno_labels)}")

    # Resultados finais
    print("\n=== RESULTADOS FINAIS ===")
    demarcadores_labels = [vertice.label for vertice in demarcadores]
    print(f"\nDemarcadores: {list(demarcadores_labels)}")

    articulacoes_labels = [vertice.label for vertice in articulacoes]
    print(f"Articulações: {list(articulacoes_labels)}")

    print("\n=== BLOCOS BICONEXOS ===")
    for i, bloco in enumerate(blocos_biconexos, start=1):
        print(f"Bloco {i}: {[v.label for v in bloco]}")

    for vertice in grafo.get_vertices():
        pai_label = pais[vertice].label if vertice in pais and pais[vertice] else "X"
        print(f"Vértice {vertice.label} (pai: {pai_label}) -> lowpt: {lowpts[vertice].label}")

In [ ]:
display_markdown("**Busca em profundidade em grafo nao conexo com determinação de blocos e articulações**")
grafo1 = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
dps_lowpt(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

## Percurso em profundidade em 'Grafo 2 - Lista de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor: 1
Vertice visitado: 3, predecessor: 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor: 3
Vertice visitado: 5, predecessor: 4
Nova aresta de retorno:  (5, 2)
Vértice 5 explorado -> lowpt: 2
Vértice 4 explorado -> lowpt: 2
Vértice 3 explorado -> lowpt: 1
Vertice visitado: 6, predecessor: 2
Nova aresta de retorno:  (6, 1)
Vertice visitado: 7, predecessor: 6
Vértice 7 explorado -> lowpt: 7
Vértice 6 explorado -> lowpt: 1
Vértice 2 explorado -> lowpt: 1
Vértice 1 explorado -> lowpt: 1
Vertice visitado: 8, predecessor: X
Vertice visitado: 9, predecessor: 8
Vertice visitado: 10, predecessor: 9
Nova aresta de retorno:  (10, 8)
Vértice 10 explorado -> lowpt: 8
Vértice 9 explorado -> lowpt: 8
Vertice visitado: 11, predecessor: 8
Vértice 11 explorado -> lowpt: 11
Vértice 8 explorado -> lowpt: 8
Arestas de retorno: [(6, 1), (10, 8), (3, 1), (5, 2)]



In [ ]:
display_markdown("**Busca em profundidade em grafo conexo com determinação de blocos e articulações**")
grafo1 = MatrizAdjacencia("Grafo 1 - Matriz de Adjacencia", "./data/GRAFO_1.txt")
dps_lowpt(grafo=grafo1, label_vertice_inicial='a', ordenado=True)

## Percurso em profundidade em 'Grafo 1 - Matriz de Adjacencia'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor: a
Vertice visitado: c, predecessor: b
Vertice visitado: d, predecessor: c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor: d
Vértice h explorado -> lowpt: h
Vértice d explorado -> lowpt: b
Vertice visitado: e, predecessor: c
Nova aresta de retorno:  ('e', 'b')
Vértice e explorado -> lowpt: b
Vertice visitado: i, predecessor: c
Vértice i explorado -> lowpt: i
Vértice c explorado -> lowpt: b
Vertice visitado: f, predecessor: b
Vértice f explorado -> lowpt: f
Vértice b explorado -> lowpt: b
Vértice a explorado -> lowpt: a
Arestas de retorno: [('d', 'b'), ('e', 'b')]

=== RESULTADOS FINAIS ===

Demarcadores: ['a', 'c', 'f', 'i', 'h', 'b']
Articulações: ['b', 'd', 'c']

=== BLOCOS BICONEXOS ===
Bloco 1: ['h', 'd']
Bloco 2: ['i', 'c']
Bloco 3: ['e', 'b', 'd', 'c']
Bloco 4: ['f', 'b']
Bloco 5: ['b', 'a']
Vértice a (pai: X) -> lowpt: a
Vérti

## DIGRAFOS

---



### **Item 16** - *Representação do Digrafo a partir da Matriz de Adjacências*

Para implementar essa função, foi seguido o procedimento apresentado em sala de aula para construção de matrizes de adjacência de dígrafos. Onde o programa realiza a leitura dos arquivos contendo as arestas de cada dígrafo, identifica automaticamente os vértices e constrói a matriz de adjacência conforme a convenção teórica:

*   1, indica uma aresta (origem -> destino)
* 0, indica a ausência

A implementação foi dividida em três etapas:
1.   Leitura das arestas
2.   Função da construção da matriz de adjacência
3.   Impressão da Matriz de Adjacência

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Sáida esperada
Impressão da matriz de adjacência correspondente
```
    a  b  c  d
a   0  1  0  0
b   0  0  1  0
c   0  0  0  0
d   1  0  0  0
```

#### Função de leitura de arquivo

In [13]:
import requests

def ler_grafo(caminho_arquivo):
    """
    Lê um arquivo de grafo de uma URL ou caminho local (via requests).
    - A primeira linha pode conter a quantidade de arestas ou vértices (será
    ignorada se for numérica).
    - As linhas seguintes devem ter o formato "origem,destino".
    """
    arestas = []
    vertices = []

    # Lê via requests
    resposta = requests.get(caminho_arquivo)
    linhas = [linha.strip() for linha in resposta.text.splitlines() if linha.strip()]

    # Ignora a primeira linha se for numérica
    if linhas and linhas[0].replace(" ", "").isdigit():
        linhas = linhas[1:]

    for linha in linhas:
        if ',' in linha:
            origem, destino = [p.strip() for p in linha.split(',')]
            # converte para int se for numérico
            origem = int(origem) if origem.isdigit() else origem
            destino = int(destino) if destino.isdigit() else destino

            arestas.append((origem, destino))

            if origem not in vertices:
                vertices.append(origem)
            if destino not in vertices:
                vertices.append(destino)
        else:
            print(f"Erro na linha: {linha}")

    return vertices, arestas

#### Função da construção da matriz de adjacência

In [7]:
def matriz_adjacencia(arestas):
    """
    Cria a estrutura de um grafo direcionado e gera sua matriz de adjacência
    a partir de uma lista de arestas. Caso existão vértices duplicados, eles são
    removidos e se todos forem numéricos, são ordenados.
    """
    grafo = {}
    grafo['arestas'] = arestas

    # Remove vértices duplicados e ordena
    vertices_ordenados = sorted({v for aresta in arestas for v in aresta})

    # Se todos os vértices são números, ordena numericamente
    if all(str(v).isdigit() for v in vertices_ordenados):
        grafo['vertices'] = sorted([int(v) for v in vertices_ordenados])
    else:
        grafo['vertices'] = sorted(vertices_ordenados)

    # Mapeia vértices para índices
    grafo['indice_vertice'] = {v: i for i, v in enumerate(grafo['vertices'])}
    grafo['labels_vertices'] = {i: v for i, v in enumerate(grafo['vertices'])}

    # Cria matriz de adjacência
    n = len(grafo['vertices'])
    matriz = [[0] * n for _ in range(n)]
    for origem, destino in arestas:
        i = grafo['indice_vertice'][origem]
        j = grafo['indice_vertice'][destino]
        matriz[i][j] += 1

    grafo['matriz'] = matriz
    print("Matriz de Adjacência")
    return grafo


#### Impressão da Matriz

In [8]:
def matriz_imprimir(grafo, matriz=None):
    """
    Exibe a matriz de adjacência de forma formatada. Onde imprime os vértices
    como cabeçalho e as linhas da matriz. Garante que todos os vértices sejam
    exibidos como strings.
    """
    if matriz is None:
        matriz = grafo['matriz']
    vertices = grafo['vertices']

    # Garante que todos os vértices sejam strings ao imprimir
    vertices_str = [str(v) for v in vertices]

    # Cabeçalho
    print("    " + " ".join(vertices_str))
    for i, linha in enumerate(matriz):
        print(f"{vertices_str[i]:>3} " + " ".join(map(str, linha)))
    print()



#### Testes:

In [17]:
print("Grafo 0")
vertices0, arestas0 = ler_grafo("https://raw.githubusercontent.com/SamuelMoraisdRS/Grafos-unidade1/refs/heads/main/data/DIGRAFO_0.txt")
graf0 = matriz_adjacencia(arestas0)
matriz_imprimir(graf0, None)

print("Grafo 1")
vertices1, arestas1 = ler_grafo("https://raw.githubusercontent.com/SamuelMoraisdRS/Grafos-unidade1/refs/heads/main/data/DIGRAFO1.txt")
graf1 = matriz_adjacencia(arestas1)
matriz_imprimir(graf1)

print("Grafo 2")
vertices2, arestas2 = ler_grafo("https://raw.githubusercontent.com/SamuelMoraisdRS/Grafos-unidade1/refs/heads/main/data/DIGRAFO2.txt")
graf2 = matriz_adjacencia(arestas2)
matriz_imprimir(graf2)

print("Grafo 3")
vertices3, arestas3 = ler_grafo("https://raw.githubusercontent.com/SamuelMoraisdRS/Grafos-unidade1/refs/heads/main/data/DIGRAFO3.txt")
graf3 = matriz_adjacencia(arestas3)
matriz_imprimir(graf3)

Grafo 0
Matriz de Adjacência
    a b c d e f g h
  a 0 1 0 1 0 0 0 0
  b 0 0 1 0 1 0 0 0
  c 1 0 0 0 0 1 0 0
  d 0 0 0 0 1 0 0 0
  e 0 0 0 0 0 1 0 0
  f 0 0 0 0 0 0 1 0
  g 0 0 0 0 0 0 0 1
  h 0 0 0 0 0 0 0 0

Grafo 1
Matriz de Adjacência
    1 2 3 4 5 6 7 8 9 10 11 12 13
  1 0 1 0 0 0 0 0 0 0 0 0 0 0
  2 0 0 1 0 0 0 0 0 0 0 0 0 0
  3 1 0 0 1 0 0 0 0 0 0 0 0 0
  4 0 0 0 0 1 0 0 0 0 0 0 0 0
  5 0 0 0 0 0 1 0 1 0 0 0 0 0
  6 0 0 0 0 0 0 1 0 0 0 0 0 0
  7 0 0 0 0 0 1 0 0 1 0 0 0 0
  8 0 0 0 1 0 0 0 0 0 1 0 0 0
  9 0 0 0 0 0 0 0 1 0 0 0 0 0
 10 0 0 0 0 0 0 0 0 0 0 0 0 0
 11 0 0 0 0 0 0 0 0 0 0 0 1 0
 12 0 0 0 0 0 0 0 0 0 0 0 0 1
 13 0 0 0 0 0 0 0 0 0 0 0 1 0

Grafo 2
Matriz de Adjacência
    1 2 3 4 5 6 7 8 9 10 11 12 13
  1 0 1 0 0 0 0 0 0 0 0 0 0 0
  2 0 0 1 0 0 0 0 0 0 0 0 0 0
  3 1 0 0 1 0 0 0 0 0 0 0 0 0
  4 0 0 0 0 1 0 0 0 0 0 0 0 0
  5 0 0 0 0 0 1 0 1 0 0 0 0 0
  6 0 0 0 0 0 0 1 0 0 0 0 0 0
  7 0 0 0 0 0 1 0 0 1 0 0 0 0
  8 0 0 0 1 0 0 0 0 0 1 0 0 0
  9 0 0 0 0 0 0 0 1 0 0 0 0 0
 10

### **Item 17** - Representação do Digrafo a partir da Matriz de Incidência.



Para implementar essa função, foi utilizado o procedimento apresentado em sala de aula para construção e análise de matrizes de incidência de dígrafos.
O programa lê arquivos contendo as arestas de cada dígrafo, identifica seus vértices e constrói a matriz de incidência seguindo a convenção teórica:

- −1 indica o vértice de origem da aresta;
- +1 indica o vértice de destino; e
- 0 indica ausência de incidência;

A partir dessa matriz, o código reconstrói a lista de adjacência e a matriz de adjacência correspondentes, permitindo visualizar as diferentes representações do mesmo dígrafo.
Por fim, as três estruturas — lista de adjacência, matriz de incidência e matriz de adjacência — são exibidas no formato mostrado nos slides, facilitando a comparação entre as representações e a compreensão das relações entre vértices e arestas.

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada
Exibição das três representações do grafo:


*  Matriz de Incidência


```
      a    b    c    d
a1   -1    1    0    0
a2    0   -1    1    0
a3    1    0    0   -1
```

*  Lista de Adjacência


```
a → ['b']
b → ['c']
d → ['a']
```


* Matriz de Adjacência


```
    a  b  c  d
a   0  1  0  0
b   0  0  1  0
c   0  0  0  0
d   1  0  0  0

```

#### Leitura do Digrafo, contrução da matriz de Incidência e tranformação de matriz de Incidência para matriz e lista de adjacência

In [ ]:
import numpy as np
import pandas as pd

def ler_digrafo_txt(caminho_arquivo):
    """
    Lê um arquivo de texto contendo as arestas de um dígrafo.
    - A primeira linha pode conter a quantidade de arestas ou vértices (será ignorada se for um número).
    - As linhas seguintes devem ter o formato "origem,destino".
    Retorna:
      - lista de vértices
      - lista de arestas (pares ordenados)
    """
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        linhas = [linha.strip() for linha in f if linha.strip()]

    # Ignora a primeira linha se for numérica
    if linhas and linhas[0].replace(" ", "").isdigit():
        linhas = linhas[1:]

    arestas = []
    vertices = []

    for linha in linhas:
        if ',' in linha:
            origem, destino = [p.strip() for p in linha.split(',')]
            arestas.append((origem, destino))
            if origem not in vertices:
                vertices.append(origem)
            if destino not in vertices:
                vertices.append(destino)

    return vertices, arestas


def construir_matriz_incidencia(vertices, arestas):
    """
    Constrói a matriz de incidência no formato teórico:
    - Linhas = arestas
    - Colunas = vértices
    Convenção:
      -1 → vértice é origem da aresta
      +1 → vértice é destino da aresta
       0 → não é incidente
    """
    n = len(vertices)
    m = len(arestas)
    indice = {v: i for i, v in enumerate(vertices)}

    # Matriz com linhas = arestas, colunas = vértices
    M = np.zeros((m, n), dtype=int)

    for i, (origem, destino) in enumerate(arestas):
        M[i, indice[origem]] = -1
        M[i, indice[destino]] = 1

    return M

def digrafo_a_partir_matriz_incidencia(M, vertices):
    """
    Reconstrói a lista e a matriz de adjacência a partir da
    matriz de incidência no formato teórico (linhas = arestas, colunas = vértices).
    """
    m, n = M.shape  # m = arestas, n = vértices
    A = np.zeros((n, n), dtype=int)
    lista_adj = {v: [] for v in vertices}

    for i in range(m):  # percorre as arestas (linhas)
        origens = np.where(M[i, :] == -1)[0]
        destinos = np.where(M[i, :] == 1)[0]

        for u in origens:
            for v in destinos:
                A[u, v] += 1
                lista_adj[vertices[u]].append(vertices[v])

    return lista_adj, A

#### Funções de exibição

In [ ]:
def mostrar_matriz_incidencia(M, vertices):
    """
    Mostra a matriz de incidência no formato passado nas aulas:
    - Linhas = arestas (a1, a2, a3, ...)
    - Colunas = vértices (v1, v2, v3, ...)
    """
    print("\nMatriz de Incidência:")
    print("      ", end="")
    for v in vertices:
        print(f"{v:^5}", end="")
    print()

    for i in range(M.shape[0]):
        print(f"a{i+1:>2}  ", end="")
        for j in range(M.shape[1]):
            print(f"{M[i, j]:^5}", end="")
        print()

def mostrar_lista_adjacencia(lista_adj, vertices):
    print("\nLista de Adjacência:")
    for v in vertices:
        print(f"{v} → {lista_adj[v]}")

def mostrar_matriz_adjacencia(matriz_adj, vertices):
    print("\nMatriz de Adjacência:")

    # Cabeçalho das colunas (vértices)
    print("     ", end="")
    for v in vertices:
        print(f"{v:^5}", end="")
    print()

    # Linhas com os vértices
    for i, v in enumerate(vertices):
        print(f"{v:>5} ", end="")
        for j in range(len(vertices)):
            print(f"{matriz_adj[i][j]:^5}", end="")
        print()

In [ ]:
arquivos = [
    "DIGRAFO_0.txt",
    "DIGRAFO1.txt",
    "DIGRAFO2.txt",
    "DIGRAFO3.txt",
]

for arq in arquivos:
    try:
        print("\n=== " + arq + " ===\n")
        vertices, arestas = ler_digrafo_txt("data/" + arq)

        M = construir_matriz_incidencia(vertices, arestas)
        lista_adj, matriz_adj = digrafo_a_partir_matriz_incidencia(M, vertices)

        mostrar_lista_adjacencia(lista_adj, vertices)
        mostrar_matriz_incidencia(M, vertices)
        mostrar_matriz_adjacencia(matriz_adj, vertices)
    except FileNotFoundError:
        print(f"Arquivo não encontrado.")


=== DIGRAFO_0.txt ===


Lista de Adjacência:
a → ['b', 'd']
b → ['c', 'e']
d → ['e']
c → ['a', 'f']
e → ['f']
f → ['g']
g → ['h']
h → []

Matriz de Incidência:
        a    b    d    c    e    f    g    h  
a 1   -1    1    0    0    0    0    0    0  
a 2   -1    0    1    0    0    0    0    0  
a 3    0   -1    0    1    0    0    0    0  
a 4    0   -1    0    0    1    0    0    0  
a 5    1    0    0   -1    0    0    0    0  
a 6    0    0    0   -1    0    1    0    0  
a 7    0    0   -1    0    1    0    0    0  
a 8    0    0    0    0   -1    1    0    0  
a 9    0    0    0    0    0   -1    1    0  
a10    0    0    0    0    0    0   -1    1  

Matriz de Adjacência:
       a    b    d    c    e    f    g    h  
    a   0    1    1    0    0    0    0    0  
    b   0    0    0    1    1    0    0    0  
    d   0    0    0    0    1    0    0    0  
    c   1    0    0    0    0    1    0    0  
    e   0    0    0    0    0    1    0    0  
    f   0    0    0    0    

### **Item 18** (Opcional) - *Determinação do Grafo subjacente*

Para implementar essa função, foi seguido o procedimento visto em sala de aula para construção do grafo subjacente de um dígrafo.
O grafo subjacente (ou não direcionado) é obtido a partir de um dígrafo, removendo-se a orientação das arestas, ou seja, cada aresta direcionada é substituída por uma conexão bidirecional entre os mesmos vértices.
#### Entrada esperada
Matriz de adjacência do dígrafo
```
    a  b  c
a   0  1  0
b   0  0  1
c   0  0  0
```

#### Sáida esperada
Exibição do grafo subjacente


```
    a  b  c
a   0  1  0
b   1  0  1
c   0  1  0
```




#### Função de Grafo Subjacente

In [18]:
def grafo_sub(grafo):
  """
  Gera o grafo subjacente (não direcionado) a partir da matriz de adjacência
  de um dígrafo, removendo a orientação das arestas.
  """
  # defini o tamanho do grafo e cria a estrutura da matriz
  n = len(grafo['vertices'])
  subjacente = [[0]*n for _ in range(n)]
  matriz = grafo['matriz']

  # O laço percorrer todos os pares de vertices e verifica se existe aresta no digrafo, se existir, atribui 1 no grafo subjacente, com conexão bidirecional.
  for i in range(n):
    for j in range(n):
      if matriz[i][j] != 0 or matriz[j][i] != 0:
        subjacente[i][j] = 1
        subjacente[j][i] = 1

  print("Subgrafo Subjacente:")
  return subjacente

#### Impressão da Matriz Subjacente

In [19]:
grafo_sub = grafo_sub(graf0)
matriz_imprimir(graf0, grafo_sub)

Subgrafo Subjacente:
    a b c d e f g h
  a 0 1 1 1 0 0 0 0
  b 1 0 1 0 1 0 0 0
  c 1 1 0 0 0 1 0 0
  d 1 0 0 0 1 0 0 0
  e 0 1 0 1 0 1 0 0
  f 0 0 1 0 1 0 1 0
  g 0 0 0 0 0 1 0 1
  h 0 0 0 0 0 0 1 0



### **Item 19** - *Busca em largura*

A função busca_largura realiza o percurso em largura em um grafo representado como matriz de adjacência. Ela percorre todos os vértices, garantindo que componentes desconexos também sejam visitados.

* Mantém a ordem de visita dos vértices.

* Armazena o predecessor de cada vértice no percurso.

* Ordena os vizinhos por rótulo (alfabética ou numérica) para garantir consistência.

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Saída esperada
Busca em Largura do digrafo


```
Ordem de visita: a → b → c → d
Predecessores:
a ← None
b ← a
c ← b
d ← None

```

In [26]:
from collections import deque

def busca_largura(grafo, origem=None):
    """
    Realiza a busca em largura (BFS) em um grafo representado como dicionário com:
    - 'matriz': matriz de adjacência
    - 'vertices': lista de rótulos dos vértices
    Retorna a ordem de visita e o predecessor de cada vértice.
    """

    # Extrai a matriz de adjacência e a lista de vértices
    matriz = grafo['matriz']
    vertices = grafo['vertices']
    n = len(vertices)

    visitados = [False] * n
    predecessor = [None] * n
    ordem_visita = []

    # Percorre todos os vértices para garantir que componentes desconexos sejam visitados
    for origem_index in range(n):
        if not visitados[origem_index]:
            fila = deque([origem_index])
            visitados[origem_index] = True

            # Processa a fila até que todos os vértices conectados sejam visitados
            while fila:
                u = fila.popleft()
                ordem_visita.append(vertices[u])

                # Lista de vizinhos não visitados
                vizinhos = [v for v in range(n) if matriz[u][v] != 0 and not visitados[v]]
                # Ordena os vizinhos por rótulo (alfabética ou numérica) para consistência
                vizinhos.sort(key=lambda v: int(vertices[v]) if vertices[v].isdigit() else vertices[v])

                # Processa cada vizinho não visitado
                for v in vizinhos:
                    predecessor[v] = vertices[u]
                    visitados[v] = True
                    fila.append(v)


    print("Ordem de visita:", " → ".join(ordem_visita))
    print("Predecessores:")
    for i, p in enumerate(predecessor):
        if p is not None:
            print(f"{vertices[i]} ← {p}")
        else:
            print(f"{vertices[i]} ← None")

    return ordem_visita, predecessor


In [27]:
ordem, predecessor = busca_largura(graf0)

Ordem de visita: a → b → d → c → e → f → g → h
Predecessores:
a ← None
b ← a
c ← b
d ← a
e ← b
f ← c
g ← f
h ← g


### **Item 20** - *Busca em profundidade, com determinação de profundidade de entrada e de saída de cada vértice, e arestas de árvore, retorno, avanço e cruzamento*

Para implementar essa função, foi utilizado o algoritmo apresentado em sala de aula, conforme os slides teóricos sobre busca em profundidade (DFS) e classificação de arestas em dígrafos.

O procedimento consiste em ler arquivos que contêm os dados de cada dígrafo, construir a lista de adjacência correspondente e, a partir dela, executar o algoritmo DFS para explorar o grafo e classificar as arestas em quatro tipos distintos:

* Aresta de árvore – conecta um vértice a um novo vértice descoberto durante a busca;
* Aresta de retorno – conecta um vértice a um ancestral na árvore de busca;

* Aresta de avanço – conecta um vértice a um descendente já totalmente processado;

* Aresta de cruzamento – conecta vértices pertencentes a diferentes subárvores de busca.

#### Entrada esperada
Arquivo do tipo .txt com as arestas do grafo, no formato:
```
a,b
b,c
d,a
```

#### Sáida esperada
Impressão dos tempos de descoberta e término de cada vértice, além da classificação das arestas conforme o tipo identificado pelo algoritmo.

```
# === DIGRAFO1.txt ===

Tempos de descoberta e término:
a: d=1, f=8
b: d=2, f=3
c: d=4, f=5
d: d=6, f=7

Arestas classificadas:
(a -> b): árvore
(b -> c): avanço
(d -> a): cruzamento

```

#### Função de busca em profundidade e Função de execução da BFS

In [1]:

def ler_digrafo(caminho):
    """
    Lê um arquivo com os dados dos dígrafos.
    Retorna uma lista de adjacência {u: [v1, v2, ...]}.
    """
    adj = {}
    with open(caminho, 'r', encoding='utf-8') as f:
        linhas = [l.strip() for l in f.readlines() if l.strip()]
        _ = linhas[0]
        for linha in linhas[1:]:
            u, v = linha.split(',')
            u, v = u.strip(), v.strip()
            if u not in adj:
                adj[u] = []
            adj[u].append(v)
            if v not in adj:
                adj[v] = []
    return adj

def dfs_digrafo(adj):
    """
  Executa busca em profundidade em dígrafo e classifica arestas.
    Entrada: Lista de Adjacência {u: [v1, v2, ...]}
    Saída:
      - descoberta: Dicionário que mapeia o vértice ao tempo que foi descoberto;
      - termino: Dicionário que mapeia o vértice ao tempo que terminou;
      - pai: Mapeia cada vértice ao seu pai;
      - tipo_aresta: Mapeia cada aresta (u, v) ao seu tipo
    """
    # WHITE: Não processado, GRAY: Em processamento, BLACK: Processado
    WHITE, GRAY, BLACK = 0, 1, 2
    cor = {v: WHITE for v in adj}
    tempo = 0
    descoberta = {}
    termino = {}
    pai = {v: None for v in adj}
    tipo_aresta = {}

  #  Percorrer profundamente todos os vértices alcançáveis a partir de u,
  #  atualizando:
  #  os tempos de descoberta e término,
  #  a cor de cada vértice,
  #  e classificando as arestas.
    def dfs_visit(u):
        nonlocal tempo
        cor[u] = GRAY
        tempo += 1
        descoberta[u] = tempo
        for v in adj[u]:
            if cor[v] == WHITE:
                tipo_aresta[(u, v)] = "árvore"
                pai[v] = u
                dfs_visit(v)
            elif cor[v] == GRAY:
                tipo_aresta[(u, v)] = "retorno"
            else:  # cor[v] == BLACK
                if descoberta[u] < descoberta[v]:
                    tipo_aresta[(u, v)] = "avanço"
                else:
                    tipo_aresta[(u, v)] = "cruzamento"
        cor[u] = BLACK
        tempo += 1
        termino[u] = tempo

    for v in adj:
        if cor[v] == WHITE:
            dfs_visit(v)

    return descoberta, termino, pai, tipo_aresta

def executar_dfs_em_arquivo(caminho):
    """
    Executa busca em profundidade no dígrafo de um arquivo informado.
    """
    print(f"\n=== {caminho} ===")
    adj = ler_digrafo(caminho)
    disc, fini, pai, tipo = dfs_digrafo(adj)

    print("\nTempos de descoberta e término:")
    for v in sorted(disc, key=lambda x: disc[x]):
        print(f"{v}: d={disc[v]}, f={fini[v]}")

    print("\nArestas classificadas:")
    for (u, v), t in tipo.items():
        print(f"({u} -> {v}): {t}")

# ---- Executar nos arquivos fornecidos ----
arquivos = [
    "DIGRAFO_0.txt",
    "DIGRAFO1.txt",
    "DIGRAFO2.txt",
    "DIGRAFO3.txt",
]

for arq in arquivos:
    try:
        executar_dfs_em_arquivo("data/" + arq)
    except FileNotFoundError:
        print(f"Arquivo não encontrado.")



=== data/DIGRAFO_0.txt ===
Arquivo não encontrado.

=== data/DIGRAFO1.txt ===
Arquivo não encontrado.

=== data/DIGRAFO2.txt ===
Arquivo não encontrado.

=== data/DIGRAFO3.txt ===
Arquivo não encontrado.
